In [1]:
import os
import vllm
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

INFO 05-09 12:03:17 [__init__.py:239] Automatically detected platform cuda.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
                'Qwen/Qwen3-30B-A3B'
            )
llm = LLM(model= 'Qwen/Qwen3-30B-A3B',
          gpu_memory_utilization=0.95,
          max_model_len=32768,
          max_num_seqs=50,
          tensor_parallel_size=1)

INFO 05-09 12:03:27 [config.py:717] This model supports multiple tasks: {'reward', 'score', 'generate', 'classify', 'embed'}. Defaulting to 'generate'.
INFO 05-09 12:03:27 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 05-09 12:03:29 [core.py:58] Initializing a V1 LLM engine (v0.8.5.post1) with config: model='Qwen/Qwen3-30B-A3B', speculative_config=None, tokenizer='Qwen/Qwen3-30B-A3B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, collec

Loading safetensors checkpoint shards:   0% Completed | 0/16 [00:00<?, ?it/s]


INFO 05-09 12:03:42 [loader.py:458] Loading weights took 11.08 seconds
INFO 05-09 12:03:42 [gpu_model_runner.py:1347] Model loading took 56.8814 GiB and 12.234741 seconds


In [ ]:
system_prompt = """
You are an intelligent information extraction system designed to process text extracted from invoices using OCR (such as Tesseract).
Your role is to extract specific pieces of information from the text with a high degree of accuracy.
This information is used in downstream systems, so precision is critical. We depend on your output.

You will receive raw invoice text and are expected to extract values for the following predefined keys:

- "Supplier": The person or organization that supplies a product or service.
- "Customer": The individual or entity receiving the product or service, typically involved in a commercial transaction.
- "Invoice Number": A unique identifier assigned to the invoice, used for tracking and accounting purposes.

You must follow these rules:
- Only extract information that is explicitly mentioned in the input text.
- Do not guess or infer values that are missing or unclear.
- If a value is not present in the text, return "None" for that field.
- Do not include any additional fields, comments, or explanations in your output.

Your response must be a valid JSON object containing only the keys listed above and their corresponding values.
"""
user_prompt = """Please extract the relevant information from the following OCR-parsed invoice text. Follow the instructions provided in the system prompt and return the results as a JSON object.

Text:
{context}"""

In [ ]:
file_names = sorted(os.listdir('two-docs'))
file_names = file_names*50
batch_size = 30
num_batches = len(file_names) // batch_size

if len(file_names) % batch_size != 0:
    num_batches += 1

dataset = []

for i in range(num_batches):
    batch = {
        "file_name": [],
        "prompt": []
    }
    document_names = file_names[i * batch_size : (i + 1) * batch_size]

    for document_name in document_names:
        text_file_path =  f"two-docs/{document_name}"
        with open(text_file_path, "r") as f:
            text = f.read()
        messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt.format(context = text)},
            ]
        tokens = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, enable_thinking=False) # <|begin_of_text|> is added
        prompt = tokenizer.decode(tokens[1:]) # remove <|begin_of_text|> 
        batch['file_name'].append(document_name)
        batch['prompt'].append(prompt)

    dataset.append(batch)


In [ ]:
results = []
sampling_params = SamplingParams(temperature=0, max_tokens=512)
for batch in dataset:
    file_names = batch['file_name']
    input_prompts = batch['prompt']
    outputs = llm.generate(input_prompts, sampling_params)
    for i, output in enumerate(outputs):
        results.append((file_names[i], output.outputs[0].text))

Processed prompts:   0%|          | 0/50 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/50 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
with open('qwen-result-batch-30.txt', "w") as f:
    for file, res in results:
        print(file)
        print(res)
        print()
        
        f.write(f"{'*' * 80}\nFile name: {file}\n{res}\n")


1_1.txt
{
  "Supplier": "Microsoft",
  "Customer": "Kate Moss",
  "Invoice Number": "1_1"
}

2_5.txt
{
  "Supplier": "MICK GORDON",
  "Customer": "Filip Forsberg",
  "Invoice Number": "None"
}

1_1.txt
{
  "Supplier": "Microsoft",
  "Customer": "Kate Moss",
  "Invoice Number": "1_1"
}

2_5.txt
{
  "Supplier": "MICK GORDON",
  "Customer": "Filip Forsberg",
  "Invoice Number": "None"
}

1_1.txt
{
  "Supplier": "Microsoft",
  "Customer": "Kate Moss",
  "Invoice Number": "1_1"
}

2_5.txt
{
  "Supplier": "MICK GORDON",
  "Customer": "Filip Forsberg",
  "Invoice Number": "None"
}

1_1.txt
{
  "Supplier": "Microsoft",
  "Customer": "Kate Moss",
  "Invoice Number": "1_1"
}

2_5.txt
{
  "Supplier": "MICK GORDON",
  "Customer": "Filip Forsberg",
  "Invoice Number": "None"
}

1_1.txt
{
  "Supplier": "Microsoft",
  "Customer": "Kate Moss",
  "Invoice Number": "1_1"
}

2_5.txt
{
  "Supplier": "MICK GORDON",
  "Customer": "Filip Forsberg",
  "Invoice Number": "None"
}

1_1.txt
{
  "Supplier": "Micro